# THE IMPACT OF COVID-19 ON THE UK BUSINESS ECONOMY PATTERN

In [3]:
# we need to install some packages (remove # to install)

# import sys
# !{sys.executable} -m pip install folium
# !{sys.executable} -m pip install geopandas
# !{sys.executable} -m pip install dash
# !{sys.executable} -m pip install dash_core_components
# !{sys.executable} -m pip install dash_html_components

# Libraries to work with numbers
import datetime as dt
import numpy as np 
import pandas as pd
from scipy import stats

# Libraries to plot
import os
import re
import folium
import geopandas as gpd
import missingno as msno
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt

# Libraries to build dashboard
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from folium.plugins import MarkerCluster, HeatMap

import warnings
warnings.filterwarnings("ignore")

/var/folders/ws/thjk0xtn2878tj62hhzdv2d00000gn/T/ipykernel_8977/476662553.py:29: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/ws/thjk0xtn2878tj62hhzdv2d00000gn/T/ipykernel_8977/476662553.py:30: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [4]:
# Get data 
part1 = pd.read_csv("BasicCompanyData-2022-07-01-part1_7.csv", low_memory=False, na_values=np.nan)
part2 = pd.read_csv("BasicCompanyData-2022-07-01-part2_7.csv", low_memory=False, na_values=np.nan)
part3 = pd.read_csv("BasicCompanyData-2022-07-01-part3_7.csv", low_memory=False, na_values=np.nan)
part4 = pd.read_csv("BasicCompanyData-2022-07-01-part4_7.csv", low_memory=False, na_values=np.nan)
part5 = pd.read_csv("BasicCompanyData-2022-07-01-part5_7.csv", low_memory=False, na_values=np.nan)
part6 = pd.read_csv("BasicCompanyData-2022-07-01-part6_7.csv", low_memory=False, na_values=np.nan)
part7 = pd.read_csv("BasicCompanyData-2022-07-01-part7_7.csv", low_memory=False, na_values=np.nan)

# Combine data
frames = [part1, part2, part3, part4, part5, part6, part7]
df = pd.concat(frames)
df.head()

,CompanyName,CompanyNumber,RegAddress.CareOf,RegAddress.POBox,RegAddress.AddressLine1,RegAddress.AddressLine2,RegAddress.PostTown,RegAddress.County,RegAddress.Country,RegAddress.PostCode,...,PreviousName_7.CONDATE,PreviousName_7.CompanyName,PreviousName_8.CONDATE,PreviousName_8.CompanyName,PreviousName_9.CONDATE,PreviousName_9.CompanyName,PreviousName_10.CONDATE,PreviousName_10.CompanyName,ConfStmtNextDueDate,ConfStmtLastMadeUpDate
0,! LTD,08209948,NaN,NaN,METROHOUSE 57 PEPPER ROAD,HUNSLET,LEEDS,YORKSHIRE,NaN,LS10 2RU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25/09/2022,11/09/2021
1,!? LTD,11399177,NaN,NaN,THE STUDIO HATHERLOW HOUSE,HATHERLOW,ROMILEY,NaN,UNITED KINGDOM,SK6 3DY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19/06/2023,05/06/2022
2,!BIG IMPACT GRAPHICS LIMITED,11743365,NaN,NaN,372 OLD STREET,335 ROSDEN HOUSE,LONDON,NaN,UNITED KINGDOM,EC1V 9LT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/01/2023,27/12/2021
3,!GOBERUB LTD,13404790,NaN,NaN,13 SOUTHMILL ROAD,NaN,BISHOP'S STORTFORD,HERTFORDSHIRE,ENGLAND,CM23 3FR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30/05/2023,16/05/2022
4,!NFOGENIE LTD,13522064,NaN,NaN,71-75 SHELTON STREET,NaN,LONDON,GREATER LONDON,UNITED KINGDOM,WC2H 9JQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03/08/2022,NaN


In [5]:
# %% Pre-processing data

# 1. Drop columns that will not be used
df.drop(['RegAddress.CareOf',
         'RegAddress.POBox',
         'Accounts.AccountRefDay',
         'Accounts.NextDueDate',
         'Returns.NextDueDate',
         'Returns.LastMadeUpDate',
         'Mortgages.NumMortCharges',
         'Mortgages.NumMortOutstanding',
         'Mortgages.NumMortPartSatisfied',
         'Mortgages.NumMortSatisfied',
         'URI',
         'PreviousName_1.CONDATE', 
         ' PreviousName_1.CompanyName',
         ' PreviousName_2.CONDATE',
         ' PreviousName_2.CompanyName',
         'PreviousName_3.CONDATE',
         ' PreviousName_3.CompanyName',
         'PreviousName_4.CONDATE',
         ' PreviousName_4.CompanyName',
         'PreviousName_5.CONDATE',
         ' PreviousName_5.CompanyName',
         'PreviousName_6.CONDATE',
         ' PreviousName_6.CompanyName',
         'PreviousName_7.CONDATE', 
         ' PreviousName_7.CompanyName',
         'PreviousName_8.CONDATE',
         ' PreviousName_8.CompanyName',
         'PreviousName_9.CONDATE',
         ' PreviousName_9.CompanyName',
         'PreviousName_10.CONDATE',
         ' PreviousName_10.CompanyName',
         "DissolutionDate"],
        axis=1, inplace=True)

# 2. Replace "." in column names
df.columns = df.columns.str.replace(' ', '') 

# 3. Rename Country of Origin to the "United Kingdom"
df['CountryOfOrigin'].replace({'ENGLAND': 'United Kingdom',
                              'GB': 'United Kingdom',
                              'WALES': 'United Kingdom',
                              'CHANNEL ISLANDS': 'United Kingdom',
                              'VIRGIN ISLANDS': 'United Kingdom',
                              'VIRGIN ISLANDS, BRITISH': 'United Kingdom',
                              'Great Britain': 'United Kingdom',
                              'ENGLAND & WALES': 'United Kingdom',
                              'ISLE OF MAN': 'United Kingdom',
                              'NORTHERN ISLAND': 'United Kingdom',
                              'GIBRALTAR': 'United Kingdom',
                              'British Virgin Island': 'United Kingdom',
                              'UNITED KINGDOM': 'United Kingdom',},
                              inplace=True)

# 4. Get uk data
uk = df[df.CountryOfOrigin == "United Kingdom"]

# 5. Clean uk data
uk = uk[~uk['RegAddress.Country'].isin([np.nan,'UNITED STATES','BRAZIL',
                                      'CANADA','LUXEMBOURG','BARBADOS',
                                      'SWITZERLAND', 'VIRGIN ISLANDS',
                                      'GUERNSEY','CHANNEL ISLANDS',
                                      'JERSEY','CAYMAN ISLANDS',
                                      'BAHAMAS','BELIZE','URUGUAY',
                                      'GIBRALTAR','MALTA','LUXEMBOURG',
                                      'BERMUDA','BARBADOS','VIRGIN ISLANDS, BRITISH',
                                      'CYMRU', 'U','COSTA RICA', 'SAINT KITTS AND NEVIS', 
                                      'BAHRAIN', 'LEBANON','PARAGUAY', 'ITALY', 'NETHERLANDS', 
                                      'GY1 2HL', 'CYPRUS','CYMRU', 'U',
                                      'COSTA RICA', 'SAINT KITTS AND NEVIS','BRITISH ISLES',
                                      'WESTERN SAHARA', 'SWEDEN','HONG KONG','TANZANIA','NEW ZEALAND',
                                      'UNITED ARAB EMIRATES', 'AZERBAIJAN','MONACO',
                                      ])]
# 6. Clean uk regions
uk['RegAddress.Country'] = uk['RegAddress.Country'].replace({'LIVERPOOL':'ENGLAND',
                                                            'NORTHERN IRELAND': 'IRELAND',
                                                            'IRELAND UNITED KINGDOM': 'IRELAND',
                                                            'CHANNEL ISLANDSSW1W 0AU': 'ENGLAND', 
                                                            'PO33 2TG': 'ENGLAND',
                                                            'SCOTLAND  UK': 'SCOTLAND',
                                                            'ENGLAND & WALES': 'ENGLAND',  # Change to "England" as all regions are in this area
                                                            'UNITED KINGDOM':'ENGLAND', 
                                                            },regex=True)

# 7. Parsing datetime
uk["IncorporationDate"] = pd.to_datetime(uk["IncorporationDate"])

# 8. Extract datetime into month and year format
uk['Year-Month'] = uk['IncorporationDate'].apply(lambda x:x.strftime('%Y-%m'))
uk['Month'] = uk['IncorporationDate'].dt.month
uk['Year'] = uk['IncorporationDate'].dt.year

# 9. Get data since covid-19 took place
uk_covid = uk[uk['Year'] >= 2020]

# Visualisation 1: Active Companies amongst Industries during Covid-19

In [102]:
# Prepare data
plot1 = uk_covid.copy()

# 1. Classify the status of the companies into "active" and "distressed"
plot1["CompanyStatus"] = plot1["CompanyStatus"].replace({"Liquidation": "Distressed",
                            'ADMINISTRATION ORDER': 'Distressed',
                            'In Administration': 'Distressed',
                            'In Administration/Administrative Receiver': 'Distressed',
                            'In Administration/Receiver Manager': 'Distressed',
                            'RECEIVERSHIP': 'Distressed',
                            'ADMINISTRATIVE RECEIVER': 'Distressed',
                            'RECEIVER MANAGER / ADMINISTRATIVE RECEIVER': 'Distressed',
                            'Active': 'Active',
                            'Active - Proposal to Strike off': 'Active',
                            'Voluntary Arrangement': 'Active',
                            'Live but Receiver Manager on at least one charge': 'Active',
                            'RECEIVER MANAGER / Distressed': 'Distressed',
                            'Distressed/Administrative Receiver': 'Distressed'},
                            regex = True)

# 2. Drop empty rows
plot1.drop(plot1.loc[plot1['SICCode.SicText_1'].isin([np.nan,'nan','None ','None','None Supplied'])].index, inplace=True)

# 3. Clean SIC code
plot1['SIC_1'] = plot1['SICCode.SicText_1'].str.slice(0,5,1)
plot1['SIC_1'] = plot1['SIC_1'].replace(' ', '') 
# Make sure all codes are numbers
plot1['SIC_1'] = pd.to_numeric(plot1['SIC_1'])

In [103]:
# 4. Using SIC_1 to assign industries
plot1['Industry'] = np.nan

# Category 1: Agriculture, Forestry and Fishing
cat_1 = list(np.arange(1110, 3221, 1))
# Category 2: Mining and Quarrying
cat_2 = list(np.arange(5102, 9901, 1))
# Category 3: Manufacturing
cat_3 = list(np.arange(10110, 33201, 1))
# Category 4: Electricity, gas, steam and air conditioning supply
cat_4 = list(np.arange(35110, 35301, 1))
# Category 5: Water supply, sewerage, waste management and remediation activities
cat_5 = list(np.arange(36000, 39001, 1))
# Category 6: Construction
cat_6 = list(np.arange(41100, 44000, 1))
# Category 7: Wholesale and retail trade; repair of motor vehicles and motorcycles
cat_7 = list(np.arange(45111, 47991, 1))
# Category 8: Transportation and storage
cat_8 = list(np.arange(49100, 53203, 1))
# Category 9: Accommodation and food service activities
cat_9 = list(np.arange(55100, 56303, 1))
# Category 10: Information and communication
cat_10 = list(np.arange(58110, 63991, 1))
# Category 11: Financial and insurance activities
cat_11 = list(np.arange(64110, 66301, 1))
# Category 12: Real estate activities
cat_12 = list(np.arange(68100, 68321, 1))
# Category 13: Professional, scientific and technical activities
cat_13 = list(np.arange(69101, 75001, 1))
# Category 14: Administrative and support service activities
cat_14 = list(np.arange(77110, 82991, 1))
# Category 15: Public administration and defence; compulsory social security
cat_15 = list(np.arange(84110, 84301, 1))
# Category 16: Education
cat_16 = list(np.arange(85100, 85601, 1))
# Category 17: Human health and social work activities
cat_17 = list(np.arange(86101, 88991, 1))
# Category 18: Arts, entertainment and recreation
cat_18 = list(np.arange(90010, 93291, 1))
# Category 19: Other service activities
cat_19 = list(np.arange(94110, 96091, 1))
# Category 20: Activities of households as employers; undifferentiated goods- and services-producing activities of households for own use
cat_20 = list(np.arange(97000, 98201, 1))
# Category 21: Activities of extraterritorial organisations and bodies
cat_21 = list(np.arange(99000, 100000, 1))

# Assign classifications to companies
plot1.loc[plot1['SIC_1'].isin(cat_1), 'Industry'] = str('Agriculture, Forestry & Fishing')
plot1.loc[plot1['SIC_1'].isin(cat_2), 'Industry'] = str('Mining & Quarrying')
plot1.loc[plot1['SIC_1'].isin(cat_3), 'Industry'] = str('Manufacturing')
plot1.loc[plot1['SIC_1'].isin(cat_4), 'Industry'] = str('Energy Supply')

plot1.loc[plot1['SIC_1'].isin(cat_5), 'Industry'] = str('Water Supply, Sewerage, Waste Management')
plot1.loc[plot1['SIC_1'].isin(cat_6), 'Industry'] = str('Construction')
plot1.loc[plot1['SIC_1'].isin(cat_7), 'Industry'] = str('Wholesale & Retail')
plot1.loc[plot1['SIC_1'].isin(cat_8), 'Industry'] = str('Transportation & Storage')

plot1.loc[plot1['SIC_1'].isin(cat_9), 'Industry'] = str('Accommodation & Food')
plot1.loc[plot1['SIC_1'].isin(cat_10), 'Industry'] = str('Information & Communication')
plot1.loc[plot1['SIC_1'].isin(cat_11), 'Industry'] = str('Financial & Insurance')
plot1.loc[plot1['SIC_1'].isin(cat_12), 'Industry'] = str('Real Estate')

plot1.loc[plot1['SIC_1'].isin(cat_13), 'Industry'] = str('Professional, Scientific & Technical')
plot1.loc[plot1['SIC_1'].isin(cat_14), 'Industry'] = str('Administrative Servives')
plot1.loc[plot1['SIC_1'].isin(cat_15), 'Industry'] = str('Administration & Security')
plot1.loc[plot1['SIC_1'].isin(cat_16), 'Industry'] = str('Education')

plot1.loc[plot1['SIC_1'].isin(cat_17), 'Industry'] = str('Health & Social Work')
plot1.loc[plot1['SIC_1'].isin(cat_18), 'Industry'] = str('Arts & Entertainment')
plot1.loc[plot1['SIC_1'].isin(cat_19), 'Industry'] = str('Other Services')
plot1.loc[plot1['SIC_1'].isin(cat_20), 'Industry'] = str('Households as Employers')
plot1.loc[plot1['SIC_1'].isin(cat_21), 'Industry'] = str('Extraterritorial Organisations')

# 5. Clean company category
plot1['CompanyCategory'].replace({'PRI/LTD BY GUAR/NSC (Private, limited by guarantee, no share capital)': 'Private Limited No Share Capital',
                                  "PRI/LBG/NSC (Private, Limited by guarantee, no share capital, use of 'Limited' exemption)": 'Private Limited Exemption',
                                  'Limited Liability Partnership': 'Limited Partnership',
                                  'Private Unlimited Company': 'Private Unlimited'},
                                  inplace = True)

In [104]:
# Get active and distressed companies
active_comp = plot1.loc[plot1['CompanyStatus'] == 'Active', :]

# Count the number of companies in each industry
size = active_comp.groupby(by=['Year-Month','Year','Industry']).size().reset_index(name='Sum')

# Plot
bar_fig = go.Figure()

bar_fig = px.bar(size, x="Industry", y="Sum", color="Industry",
                 animation_frame="Year-Month", animation_group="Industry", 
                 range_y=[0, 10000],title="<b>CHANGES OF ACTIVE COMPANIES AMONGST INDUSTRIES DURING COVID-19<b>",)

bar_fig.update_layout(font=dict(family="Futura", size=8, color="Grey"), 
                      title_x=0.5,title_font_size=20,title_font_color='black',
                      width=1000, height=600,title_font_family="Futura",
                      paper_bgcolor="#ededed")

# y axis
bar_fig.update_yaxes(title="Total Number of Companies <i>(in thousands)</i>")

# Save figure 
bar_fig.write_html('ActiveCompaniesDuringCovid-19.html')

# display figure
bar_fig.show()

In [109]:
# Plot
line_fig = go.Figure()

line_fig = px.line(size, x="Year-Month", y="Sum", color="Industry",
                    animation_group="Industry", range_y=[0, 14000],title="<b>CHANGES OF ACTIVE COMPANIES AMONGST INDUSTRIES DURING COVID-19<b>",)

line_fig.update_layout(font=dict(family="Futura", size=8, color="Grey"), 
                      title_x=0.5,title_font_size=20,title_font_color='black',
                      width=1000, height=600,title_font_family="Futura",
                      paper_bgcolor="#ededed")

# y axis
line_fig.update_yaxes(title="Total Number of Companies <i>(in thousands)</i>")

# Save figure 
line_fig.write_html('ActiveCompaniesDuringCovid-19-Line.html')

# display figure
line_fig.show()

# VISUALISATION 2: HEATMAP OF DISTRESSED BUSINESSES IN THE UK DURING COVID-19

In [16]:
# Import coordinates data
geo_df = pd.read_csv("ukpostcodes.csv")
geo_df = geo_df.loc[geo_df["postcode"].apply(lambda x: True if re.findall(r"^[a-zA-Z]+", x) else False)]
geo_df["code"] = geo_df["postcode"].apply(lambda x: re.findall(r"^[a-zA-Z]+", x)[0])

# Get the uk map
uk_bound = gpd.read_file("uk_regions.geojson")

# Get the company going distressed during pandemics
distress = plot1.loc[plot1['CompanyStatus'] == 'Distressed', :]

# Get necessary columns
distress1 = distress[['SIC_1', 'CompanyStatus', 'Industry', 'RegAddress.PostCode', 'Year', 'Year-Month']]

# Merge postcode latitude and longitude data 
distress2 = pd.merge(left=distress1.rename({'RegAddress.PostCode':'postcode'}, axis=1),
         right=geo_df, on='postcode').dropna().drop_duplicates().reset_index(drop=True).drop('id', axis=1)

# Create description columns
distress2['description'] = 'SIC category: ' + distress2['Industry'] + '\n' + 'Company Status: ' + distress2['CompanyStatus']

# View data
distress2.head()

,SIC_1,CompanyStatus,Industry,postcode,Year,Year-Month,latitude,longitude,code,description
0,62090,Distressed,Information & Communication,TR4 8UN,2020,2020-05,50.286604,-5.154590,TR,SIC category: Information & Communication\nCom...
1,64999,Distressed,Financial & Insurance,SE1 9EY,2021,2021-09,51.507644,-0.107208,SE,SIC category: Financial & Insurance\nCompany S...
2,41202,Distressed,Construction,L1 0AH,2020,2020-01,53.395947,-2.980549,L,SIC category: Construction\nCompany Status: Di...
3,47890,Distressed,Wholesale & Retail,CV5 8EF,2020,2020-03,52.407448,-1.538685,CV,SIC category: Wholesale & Retail\nCompany Stat...
4,99999,Distressed,Extraterritorial Organisations,SG4 7DH,2020,2020-09,51.955684,-0.168762,SG,SIC category: Extraterritorial Organisations\n...


In [17]:
# Plot geo heatmap
map_fig = px.density_mapbox(distress2, lat='latitude', lon='longitude', radius=6,
                        center=dict(lat=53.5500, lon=-2.4333 ), zoom=4,
                        mapbox_style="open-street-map", animation_frame='Year-Month')

map_fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))

map_fig.update_layout(font=dict(family="Futura", size=8, color="Grey"),
        title = '<br>COVID-19: UK Business Distress and Closures</br>',
        title_font_size=11,title_font_color='#747175',title_x=0.01, 
        title_y=0.98,paper_bgcolor="#ededed", height=500, width=1000,
        geo = dict(
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5),
            )

# Save figure 
map_fig.write_html('UKBusinessDistressandClosures.html')

# display figure
map_fig.show()

# VISUALISATION 3: COVID-19: Business Financial Crisis

In [58]:
# Revalue companies' status in more details
plot3 = uk.copy()

# Clean Company Status
plot3.drop(plot3.loc[plot3['CompanyStatus'].isin([np.nan,'nan','None ','None','None Supplied'])].index, inplace=True)
plot3['CompanyStatus'] = plot3['CompanyStatus'].replace({ 'Liquidation': 'Liquidation',
                                                      'ADMINISTRATION ORDER': 'In Administration',
                                                      'In Administration/Administrative Receiver': 'In Administration',
                                                      'In Administration/Receiver Manager': 'In Administration',
                                                      'RECEIVERSHIP': 'In Administration',
                                                      'ADMINISTRATIVE RECEIVER': 'In Administration',
                                                      'RECEIVER MANAGER / ADMINISTRATIVE RECEIVER': 'In Administration',
                                                      'Active': 'Active',
                                                      'On': 'Active',
                                                      'Active - Proposal to Strike off': 'Proposal to Strike off',
                                                      'Voluntary Arrangement': 'Voluntary Arrangement',
                                                      'Company Company Company Voluntary Arrangement': 'Voluntary Arrangement',
                                                      'Live but Receiver Manager on at least one charge': 'On',
                                                      'Company Company Voluntary Arrangement': 'Voluntary Arrangement',
                                                      'Administrative Receiver': 'In Administration',
                                                      'RECEIVER MANAGER / In Administration': 'In Administration',
                                                      'On': 'Active',
                                                      'RECEIVER MANAGER / In Administration': 'In Administration',
                                                      },
                                                       regex = True)

# Clean SIC code
plot3.drop(plot3.loc[plot3['SICCode.SicText_1'].isin([np.nan,'nan','None ','None','None Supplied'])].index, inplace=True)
plot3['SIC_1'] = plot3['SICCode.SicText_1'].str.slice(0,5,1)
plot3['SIC_1'] = plot3['SIC_1'].replace(' ', '') 
# Make sure all codes are numbers
plot3['SIC_1'] = pd.to_numeric(plot3['SIC_1'])

# Clean Industry
# Assign classifications to companies
plot3.loc[plot3['SIC_1'].isin(cat_1), 'Industry'] = str('Agriculture, Forestry & Fishing')
plot3.loc[plot3['SIC_1'].isin(cat_2), 'Industry'] = str('Mining & Quarrying')
plot3.loc[plot3['SIC_1'].isin(cat_3), 'Industry'] = str('Manufacturing')
plot3.loc[plot3['SIC_1'].isin(cat_4), 'Industry'] = str('Energy Supply')

plot3.loc[plot3['SIC_1'].isin(cat_5), 'Industry'] = str('Water Supply, Sewerage, Waste Management')
plot3.loc[plot3['SIC_1'].isin(cat_6), 'Industry'] = str('Construction')
plot3.loc[plot3['SIC_1'].isin(cat_7), 'Industry'] = str('Wholesale & Retail')
plot3.loc[plot3['SIC_1'].isin(cat_8), 'Industry'] = str('Transportation & Storage')

plot3.loc[plot3['SIC_1'].isin(cat_9), 'Industry'] = str('Accommodation & Food')
plot3.loc[plot3['SIC_1'].isin(cat_10), 'Industry'] = str('Information & Communication')
plot3.loc[plot3['SIC_1'].isin(cat_11), 'Industry'] = str('Financial & Insurance')
plot3.loc[plot3['SIC_1'].isin(cat_12), 'Industry'] = str('Real Estate')

plot3.loc[plot3['SIC_1'].isin(cat_13), 'Industry'] = str('Professional, Scientific & Technical')
plot3.loc[plot3['SIC_1'].isin(cat_14), 'Industry'] = str('Administrative Servives')
plot3.loc[plot3['SIC_1'].isin(cat_15), 'Industry'] = str('Administration & Security')
plot3.loc[plot3['SIC_1'].isin(cat_16), 'Industry'] = str('Education')

plot3.loc[plot3['SIC_1'].isin(cat_17), 'Industry'] = str('Health & Social Work')
plot3.loc[plot3['SIC_1'].isin(cat_18), 'Industry'] = str('Arts & Entertainment')
plot3.loc[plot3['SIC_1'].isin(cat_19), 'Industry'] = str('Other Services')
plot3.loc[plot3['SIC_1'].isin(cat_20), 'Industry'] = str('Households as Employers')
plot3.loc[plot3['SIC_1'].isin(cat_21), 'Industry'] = str('Extraterritorial Organisations')

plot3.drop(plot3.loc[plot3['Industry'].isin([np.nan,'nan','None ','None','None Supplied'])].index, inplace=True)


In [67]:
# Get the labels to plot
labels = list(plot3['Industry'].unique())
labels.extend(list(plot3['CompanyStatus'].unique()))

for index, name in enumerate(labels):
    print(index, name)

0 Wholesale & Retail
1 Manufacturing
2 Information & Communication
3 Real Estate
4 Professional, Scientific & Technical
5 Education
6 Transportation & Storage
7 Households as Employers
8 Administrative Servives
9 Health & Social Work
10 Other Services
11 Construction
12 Financial & Insurance
13 Accommodation & Food
14 Extraterritorial Organisations
15 Arts & Entertainment
16 Energy Supply
17 Agriculture, Forestry & Fishing
18 Mining & Quarrying
19 Water Supply, Sewerage, Waste Management
20 Administration & Security
21 Active
22 Liquidation
23 Proposal to Strike off
24 Voluntary Arrangement
25 In Administration


In [91]:
# Count companies in term of industry
plot3_new = pd.DataFrame(plot3.groupby(['Industry', 'CompanyStatus'])['CompanyName'].count())
# plot3_new.to_csv("CompanyStatus.csv")
plot3_new.reset_index(inplace=True)
plot3_new.head(5)

,Industry,CompanyStatus,CompanyName
0,Accommodation & Food,Active,177249
1,Accommodation & Food,In Administration,2
2,Accommodation & Food,Liquidation,467
3,Accommodation & Food,Proposal to Strike off,18519
4,Accommodation & Food,Voluntary Arrangement,43


In [95]:
# Creat "start" and "target" for sankey diagram

plot3_new['start'] = np.nan

plot3_new.loc[plot3_new['Industry'] == 'Wholesale & Retail', 'start'] = 0
plot3_new.loc[plot3_new['Industry'] == 'Manufacturing', 'start'] = 1
plot3_new.loc[plot3_new['Industry'] == 'Information & Communication', 'start'] = 2
plot3_new.loc[plot3_new['Industry'] == 'Real Estate', 'start'] = 3
plot3_new.loc[plot3_new['Industry'] == 'Professional, Scientific & Technical', 'start'] = 4
plot3_new.loc[plot3_new['Industry'] == 'Educationn', 'start'] = 5
plot3_new.loc[plot3_new['Industry'] == 'Transportation & Storage', 'start'] = 6
plot3_new.loc[plot3_new['Industry'] == 'Households as Employers', 'start'] = 7
plot3_new.loc[plot3_new['Industry'] == 'Administrative Servives', 'start'] = 8
plot3_new.loc[plot3_new['Industry'] == 'Health & Social Work', 'start'] = 9
plot3_new.loc[plot3_new['Industry'] == 'Other Services', 'start'] = 10
plot3_new.loc[plot3_new['Industry'] == 'Construction', 'start'] = 11
plot3_new.loc[plot3_new['Industry'] == 'Financial & Insurance', 'start'] = 12
plot3_new.loc[plot3_new['Industry'] == 'Accommodation & Food', 'start'] = 13
plot3_new.loc[plot3_new['Industry'] == 'Extraterritorial Organisations', 'start'] = 14
plot3_new.loc[plot3_new['Industry'] == 'Arts & Entertainment', 'start'] = 15
plot3_new.loc[plot3_new['Industry'] == 'Energy Supply', 'start'] = 16
plot3_new.loc[plot3_new['Industry'] == 'Agriculture, Forestry & Fishing', 'start'] = 17
plot3_new.loc[plot3_new['Industry'] == 'Mining & Quarrying', 'start'] = 18
plot3_new.loc[plot3_new['Industry'] == 'Water Supply, Sewerage, Waste Management', 'start'] = 19
plot3_new.loc[plot3_new['Industry'] == 'Administration & Security', 'start'] = 20

plot3_new['start'] = pd.to_numeric(plot3_new['start'])
plot3_new.drop(plot3_new.loc[plot3_new['start'].isin([np.nan,'nan','None ','None','None Supplied'])].index, inplace=True)

plot3_new.loc[plot3_new['CompanyStatus'] == 'Active', 'target'] = 21
plot3_new.loc[plot3_new['CompanyStatus'] == 'Liquidation', 'target'] = 22
plot3_new.loc[plot3_new['CompanyStatus'] == 'Proposal to Strike off', 'target'] = 23
plot3_new.loc[plot3_new['CompanyStatus'] == 'Voluntary Arrangement', 'target'] = 23
plot3_new.loc[plot3_new['CompanyStatus'] == 'In Administration', 'target'] = 25

plot3_new['target'] = pd.to_numeric(plot3_new['target'])

In [100]:
# Get data to plot
start = list(int(i) for i in round(plot3_new['start']))
target = list(int(i) for i in round(plot3_new['target']))
size = list(plot3_new['CompanyName'])

# Plot sankey diagram

# customize node colors
color_node = ['#808B96', '#EC7063', '#F7DC6F', '#48C9B0', '#AF7AC5', '#EC7063',
              '#EC7063', '#F7DC6F', '#F7DC6F', '#48C9B0', '#48C9B0', '#48C9B0', 
              '#48C9B0', '#48C9B0', '#48C9B0', '#AF7AC5', '#AF7AC5', '#AF7AC5',
              '#48C9B0']

# customize link colors
color_link = ['#EBBAB5', '#292421', '#FEF3C7', '#292421', '#A2B5CD', '#292421',
              '#CBB4D5', '#292421', '#EBBAB5', '#292421', '#EBBAB5', '#292421',
              '#FEF3C7', '#292421', '#FEF3C7', '#292421', '#A6E3D7', '#292421', 
              '#CDB38B', '#292421', '#A2B5CD', '#292421', '#A2B5CD', '#292421', 
              '#CDB38B', '#292421', '#A6E3D7', '#292421', '#CBB4D5', '#292421', 
              '#CBB4D5', '#292421']

# Create edges
link = dict(source = start, target = target, value = size, color = color_link)
node = dict(label = labels, pad = 10, thickness = 5, color = color_node)
data = go.Sankey(link = link, node = node)

# Create figure
sankey_fig = go.Figure(data)

sankey_fig.update_layout(autosize=False,
                  width=1000,
                  height=600,
                  paper_bgcolor='#E0EEEE',
                  title='<b>COVID-19: UK BUSINESS PERFORMANCE 2020-22</b>',
                  title_font_size=25,
                  title_font_family='Futura',
                  title_font_color='#3D3D3D',
                  font_color='#3D3D3D',
                  font_family='Futura',
                  font_size=12,
                 )

sankey_fig.add_annotation(dict(font=dict(color='#808080'),
                                        x=0.0,
                                        y=-0.1,
                                        showarrow=False,
                                        text='Source: '+'<a href="http://download.companieshouse.gov.uk/en_output.html">Company House</a>',
                                        textangle=0,
                                        font_family='Futura',
                                        xanchor='left',))

sankey_fig.update_layout(title_x=0.08)


# Save figure 
sankey_fig.write_html('UKBusinessPerformance.html')

# Show figure
sankey_fig.show()